In [1]:
import fiona, os, geopandas as gpd, pandas as pd

# Vertiports and corridors location

## Data preprocessing

### Area definition 

In [2]:
# individuo il path
regional_borders_emilia = "E:\Dati analisi\Limiti01012023\Reg01012023\Reg01012023_WGS84.shp"

# Leggi il file shapefile in un GeoDataFrame
regional_borders_shp = gpd.read_file(regional_borders_emilia)

# Seleziona solo la riga corrispondente alla regione "Emilia-Romagna" nella colonna "DEN_REG"
emilia_romagna_geometry = regional_borders_shp[regional_borders_shp['DEN_REG'] == 'Emilia-Romagna']

### Buildings 

In [2]:
layers_list = fiona.listlayers('E:\Dati analisi\DBMS_IGM\BO_dbsn_3J0M0f\Bologna_dbsn.gdb')

In [3]:
layers_list

['ab_cda',
 'acq_in',
 'acq_ter',
 'af_acq',
 'alveo',
 'argine',
 'ar_mar',
 'ar_nat',
 'ar_vrd',
 'a_scom',
 'attr_sp',
 'bi_ind',
 'bosco',
 'caposd',
 'cascata',
 'cl_agr',
 'cm_mon',
 'comune',
 'condot',
 'cs_mar',
 'cv_aes',
 'cv_dis',
 'cv_liv',
 'diga',
 'edifc',
 'edi_min',
 'el_acq',
 'el_fer',
 'el_fne',
 'el_fun',
 'el_idr',
 'el_met',
 'el_trv',
 'el_vms',
 'f_nter',
 'ghi_nv',
 'gz_fer',
 'gz_fun',
 'gz_met',
 'gz_trv',
 'invaso',
 'iz_str',
 'lm_am',
 'ln_btm',
 'loc_sg',
 'man_tr',
 'meta',
 'mn_ind',
 'mn_int',
 'mn_mau',
 'mu_div',
 'mu_sos',
 'nd_ele',
 'nd_idr',
 'op_por',
 'op_reg',
 'pe_uins',
 'provin',
 'ps_inc',
 'pt_btm',
 'pt_quo',
 'region',
 'scarpt',
 'sc_dis',
 'sp_acq',
 'stato',
 'sv_aer',
 'sv_fer',
 'sv_por',
 'sv_str',
 'tralic',
 'tr_ele',
 'tr_str',
 'v_rete',
 'z_tin']

In [4]:
# Specifica la cartella che contiene i file .gdb
folder_path = r'E:\Dati analisi\DBMS_IGM'

# Crea una lista vuota per contenere i GeoDataFrame dei layer "edifc" e "edi_min"
merged_layers = []

# Itera attraverso tutti i file .gdb nella cartella
for root, dirs, files in os.walk(folder_path):
    for dir in dirs:
        if dir.endswith('.gdb'):
            gdb_path = os.path.join(root, dir)
            
            # Leggi il layer "edifc" dalla Geodatabase (se esiste)
            try:
                edifc = gpd.read_file(gdb_path, layer='edifc')
                merged_layers.append(edifc)
            except ValueError:
                pass  # Il layer "edifc" non esiste in questa Geodatabase
            
            # Leggi il layer "edi_min" dalla Geodatabase (se esiste)
            try:
                edi_min = gpd.read_file(gdb_path, layer='edi_min')
                
                # Aggiungi i campi "edifc_ty" e "edifc_uso" con il valore "edi_min"
                edi_min['edifc_ty'] = 'edi_min'
                edi_min['edifc_uso'] = 'edi_min'
                
                merged_layers.append(edi_min)
            except ValueError:
                pass  # Il layer "edi_min" non esiste in questa Geodatabase

# Unisci i GeoDataFrame dei layer "edifc" e "edi_min"
if merged_layers:
    merged_data = gpd.GeoDataFrame(pd.concat(merged_layers, ignore_index=True), crs=merged_layers[0].crs)

In [5]:
# Specifica il percorso in cui desideri salvare il nuovo file shapefile
output_shp_path = r'E:\Dati analisi\DBMS_IGM\buildings_merged.shp'

# Salva il GeoDataFrame in un nuovo file shapefile
merged_data.to_file(output_shp_path, driver='ESRI Shapefile')

C:\Users\niett\AppData\Local\Temp\ipykernel_19336\3262264503.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_data.to_file(output_shp_path, driver='ESRI Shapefile')


In [6]:
# Esegui il dissolve sulla colonna "edifc_uso"
dissolved_data = merged_data.dissolve(by='edifc_uso')

In [ ]:
# Specifica il percorso in cui desideri salvare il nuovo file shapefile
output_shp_path = r'E:\Dati analisi\DBMS_IGM\buildings_merged_dissolved.shp'

# Salva il GeoDataFrame in un nuovo file shapefile
dissolved_data.to_file(output_shp_path, driver='ESRI Shapefile')

C:\Users\niett\AppData\Local\Temp\ipykernel_19336\1990260576.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dissolved_data.to_file(output_shp_path, driver='ESRI Shapefile')


### Get html table from IGM DBSN 

In [4]:
# Specifica l'URL del sito web che contiene la tabella HTML
url = "https://www.igmi.org/dbsn_supporto/dbsn/"

# Leggi la tabella HTML dal sito web
tables = pd.read_html(url)

# Itera attraverso le tabelle per trovare quella con "edifc_uso" nella prima colonna
target_table = None
for table_df in tables:
    if "edifc_uso" in table_df.iloc[:, 0].values:
        target_table = table_df
        break

In [5]:
tables_split = []
current_table_rows = []
count = 0

for index, row in target_table.iterrows():
    if not pd.isna(row.iloc[0]):
        if count > 0:
            tables_split.append(pd.DataFrame(current_table_rows, columns=target_table.columns))
            current_table_rows = []            
        
        current_table_rows.append(row)
        count += 1
        
    else:
        # Se la prima colonna è NaN, aggiungi la riga alla lista delle righe correnti
        current_table_rows.append(row)

In [6]:
# Il testo che stai cercando
text_to_find = "edifc_uso"

# Cerca la tabella con il testo nella prima colonna
selected_table = None
for table_df in tables_split:
    if text_to_find in table_df.iloc[:, 0].values:
        selected_table = table_df
        break

In [7]:
# Seleziona solo le colonne "Nome", "Definizione" e "Tipo" dalla tabella
selected_table = selected_table[["Definizione", "Tipo"]].reset_index(drop=True)

# Rinomina la colonna "Definizione" in "NuovaColonna1"
selected_table = selected_table.rename(columns={"Definizione": "edifc_uso"})

selected_table.drop(0, inplace=True)

In [8]:
selected_table

,edifc_uso,Tipo
1,01,residenziale
2,0101,abitativa
3,02,amministrativo
4,0201,municipio
5,0202,sede provincia
...,...,...
86,1202,"sede albergo, locanda"
87,1203,campeggio
88,1204,rifugio montano
89,93,non definito


In [9]:
# Salva il DataFrame in un file Excel
selected_table.to_excel(r'E:\Dati analisi\DBMS_IGM\use_description.xlsx', index=False) 

### Join dissolve buildings with use description

In [14]:
# Specifica il percorso in cui desideri salvare il nuovo file shapefile
output_shp_path = r'E:\Dati analisi\DBMS_IGM\buildings_merged_dissolved.shp'

# Leggi il file shapefile in un GeoDataFrame
dissolved_data = gpd.read_file(output_shp_path)

In [20]:
# Esegui il join tra il DataFrame e il GeoDataFrame sulla colonna "edifc_uso"
dissolved_data_join = dissolved_data.merge(selected_table, on="edifc_uso", how="inner")

In [ ]:
# Specifica il percorso in cui desideri salvare il nuovo file shapefile
output_shp_path = r'E:\Dati analisi\DBMS_IGM\buildings_merged_join.shp'

# Salva il GeoDataFrame in un nuovo file shapefile
dissolved_data_join.to_file(output_shp_path, driver='ESRI Shapefile')

In [1]:
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# Carica il tuo GeoDataFrame
gdf = gpd.read_file('Limiti01012023/Reg01012023/Reg01012023_WGS84.shp')  # Sostituisci 'tuofile.geojson' con il percorso del tuo file GeoJSON o Shapefile

In [3]:
gdf

,COD_RIP,COD_REG,DEN_REG,Shape_Leng,Shape_Area,geometry
0,1,1,Piemonte,1.333428e+06,2.538670e+10,"POLYGON ((456874.626 5145767.794, 456879.627 5..."
1,1,2,Valle d'Aosta,3.258364e+05,3.260854e+09,"POLYGON ((389358.653 5093659.158, 389433.654 5..."
2,1,3,Lombardia,1.511132e+06,2.386310e+10,"MULTIPOLYGON (((485366.070 4984899.481, 485306..."
3,2,4,Trentino-Alto Adige,8.598731e+05,1.360472e+10,"POLYGON ((743387.812 5219741.793, 743414.749 5..."
4,2,5,Veneto,1.140433e+06,1.834537e+10,"POLYGON ((768135.210 5175357.497, 768146.460 5..."
5,2,6,Friuli Venezia Giulia,7.692437e+05,7.932482e+09,"MULTIPOLYGON (((872344.457 5061894.048, 872344..."
6,1,7,Liguria,1.079642e+06,5.416152e+09,"MULTIPOLYGON (((568230.816 4874870.697, 568232..."
7,2,8,Emilia-Romagna,1.289119e+06,2.250143e+10,"MULTIPOLYGON (((618335.211 4893983.160, 618329..."
8,3,9,Toscana,1.552065e+06,2.298744e+10,"MULTIPOLYGON (((673999.896 4681200.360, 674032..."
9,3,10,Umbria,6.714160e+05,8.464223e+09,"MULTIPOLYGON (((776426.122 4834159.173, 776480..."


In [4]:
single_region = gdf[gdf['DEN_REG'] == 'Emilia-Romagna']

In [5]:
# Calcola i valori minimi e massimi delle coordinate
lat_min = single_region.geometry.bounds['miny'].min()
lon_min = single_region.geometry.bounds['minx'].min()
lat_max = single_region.geometry.bounds['maxy'].max()
lon_max = single_region.geometry.bounds['maxx'].max()

# Stampare i risultati
print("Latitudine Minima:", lat_min)
print("Longitudine Minima:", lon_min)
print("Latitudine Massima:", lat_max)
print("Longitudine Massima:", lon_max)

Latitudine Minima: 4847056.959899999
Longitudine Minima: 515693.1046000002
Latitudine Massima: 4998812.3695
Longitudine Massima: 801305.5542000001


In [6]:
# Crea oggetti Point dai valori minimi e massimi delle coordinate
min_point = Point(lon_min, lat_min)
max_point = Point(lon_max, lat_max)

# Crea un GeoDataFrame per i punti minimi e massimi
points_gdf = gpd.GeoDataFrame(geometry=[min_point, max_point], crs=gdf.crs)

# Salva il GeoDataFrame come un file Shapefile
points_gdf.to_file('min_max_points.shp')

In [8]:
single_region_crs = single_region.to_crs(4326)

In [9]:
# Calcola i valori minimi e massimi delle coordinate
lat_min = single_region_crs.geometry.bounds['miny'].min()
lon_min = single_region_crs.geometry.bounds['minx'].min()
lat_max = single_region_crs.geometry.bounds['maxy'].max()
lon_max = single_region_crs.geometry.bounds['maxx'].max()

# Stampare i risultati
print("Latitudine Minima:", lat_min)
print("Longitudine Minima:", lon_min)
print("Latitudine Massima:", lat_max)
print("Longitudine Massima:", lon_max)

Latitudine Minima: 43.7318917110636
Longitudine Minima: 9.197911727336376
Latitudine Massima: 45.13913153551511
Longitudine Massima: 12.755640592249431


In [11]:
# Crea oggetti Point dai valori minimi e massimi delle coordinate
min_point = Point(lon_min, lat_min)
max_point = Point(lon_max, lat_max)

# Crea un GeoDataFrame per i punti minimi e massimi
points_gdf = gpd.GeoDataFrame(geometry=[min_point, max_point], crs=single_region_crs.crs)

# Salva il GeoDataFrame come un file Shapefile
points_gdf.to_file('min_max_points.shp')

In [7]:
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry import Point, box

# Crea un GeoDataFrame con il box in EPSG 32632
gdf32632 = gpd.read_file('Limiti01012023/Reg01012023/Reg01012023_WGS84.shp')

# Definisci il trasformatore da EPSG 32632 a EPSG 4326
transformer = Transformer.from_crs('EPSG:32632', 'EPSG:4326')

# Trasforma il box in EPSG 4326
box4326 = gdf32632.geometry.apply(lambda geom: box(*transformer.transform(*geom.bounds)))

# Estrai i valori minimi e massimi delle coordinate in EPSG 4326
lat_min = box4326.bounds['miny'].min()
lon_min = box4326.bounds['minx'].min()
lat_max = box4326.bounds['maxy'].max()
lon_max = box4326.bounds['maxx'].max()

# Crea oggetti Point dai valori minimi e massimi delle coordinate in EPSG 4326
min_point = Point(lon_min, lat_min)
max_point = Point(lon_max, lat_max)

# Crea un GeoDataFrame per i punti minimi e massimi in EPSG 4326
points_gdf = gpd.GeoDataFrame(geometry=[min_point, max_point], crs='EPSG:4326')

# Salva il GeoDataFrame come file Shapefile
points_gdf.to_file('min_max_points_EPSG4326.shp')

# Stampa i risultati in EPSG 4326 (WGS84)
print("Latitudine Minima (EPSG 4326):", lat_min)
print("Longitudine Minima (EPSG 4326):", lon_min)
print("Latitudine Massima (EPSG 4326):", lat_max)
print("Longitudine Massima (EPSG 4326):", lon_max)

Latitudine Minima (EPSG 4326): 6.669380890553629
Longitudine Minima (EPSG 4326): 35.51244954620116
Latitudine Massima (EPSG 4326): 5220292.2928
Longitudine Massima (EPSG 4326): 1312016.1509999996


,COD_RIP,COD_REG,DEN_REG,Shape_Leng,Shape_Area,geometry
0,1,1,Piemonte,1.333428e+06,2.538670e+10,"POLYGON ((456874.626 5145767.794, 456879.627 5..."
1,1,2,Valle d'Aosta,3.258364e+05,3.260854e+09,"POLYGON ((389358.653 5093659.158, 389433.654 5..."
2,1,3,Lombardia,1.511132e+06,2.386310e+10,"MULTIPOLYGON (((485366.070 4984899.481, 485306..."
3,2,4,Trentino-Alto Adige,8.598731e+05,1.360472e+10,"POLYGON ((743387.812 5219741.793, 743414.749 5..."
4,2,5,Veneto,1.140433e+06,1.834537e+10,"POLYGON ((768135.210 5175357.497, 768146.460 5..."
5,2,6,Friuli Venezia Giulia,7.692437e+05,7.932482e+09,"MULTIPOLYGON (((872344.457 5061894.048, 872344..."
6,1,7,Liguria,1.079642e+06,5.416152e+09,"MULTIPOLYGON (((568230.816 4874870.697, 568232..."
7,2,8,Emilia-Romagna,1.289119e+06,2.250143e+10,"MULTIPOLYGON (((618335.211 4893983.160, 618329..."
8,3,9,Toscana,1.552065e+06,2.298744e+10,"MULTIPOLYGON (((673999.896 4681200.360, 674032..."
9,3,10,Umbria,6.714160e+05,8.464223e+09,"MULTIPOLYGON (((776426.122 4834159.173, 776480..."
